# Kaggle Playground - January

In [1]:
import pandas as pd 

In [5]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.tail()
test.tail()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
199995,499984,0.353856,0.677578,0.550852,0.869612,0.957635,0.255054,0.289138,0.635979,0.271399,0.282455,0.217169,0.219088,0.373261,0.272479
199996,499985,0.243209,0.135627,0.218393,0.792798,0.547639,0.433520,0.549540,0.650107,0.453787,0.459689,0.450424,0.511176,0.318334,0.395747
199997,499987,0.506973,0.683893,0.533434,0.192957,0.314381,0.358604,0.554455,0.267105,0.396101,0.445390,0.382656,0.397978,0.381235,0.369464
199998,499988,0.347870,0.553112,0.495284,0.861500,0.816914,0.298478,0.275964,0.265841,0.334250,0.252635,0.213589,0.285223,0.336772,0.388505
199999,499990,0.688188,0.358328,0.644511,0.766354,0.747591,0.727329,0.468329,0.576711,0.453124,0.381600,0.299126,0.499627,0.691078,0.668109
